In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['/home/dr_lunars/elasticsearch-7.0.0/bin/elasticsearch'],stdout=PIPE, stderr=STDOUT)
!sleep 30

In [ ]:
!/home/dr_lunars/elasticsearch-7.0.0/bin/elasticsearch-plugin install analysis-nori

In [ ]:
!/home/dr_lunars/elasticsearch-7.0.0/bin/elasticsearch-plugin install https://github.com/javacafe-project/elasticsearch-plugin/releases/download/v7.0.0/javacafe-analyzer-7.0.0.zip

In [ ]:
es_server.kill()

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['/home/dr_lunars/elasticsearch-7.0.0/bin/elasticsearch'],stdout=PIPE, stderr=STDOUT)
!sleep 30

In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200", timeout=300, max_retries=10, retry_on_timeout=True)

print(es.info())

In [ ]:
es.indices.create(index = 'document',
                  body = {
                      'settings':{
                          'analysis':{
                              'analyzer':{
                                  'my_analyzer':{
                                      "type": "custom",
                                      'tokenizer':'nori_tokenizer',
                                      'decompound_mode':'mixed',
                                      'stopwords':'_korean_',
                                      'synonyms':'_korean_',
                                      "filter": ["lowercase",
                                                 "my_shingle_f",
                                                 "nori_readingform",
                                                 "cjk_bigram",
                                                 "decimal_digit",
                                                 "stemmer",
                                                 "trim"]
                                  },
                                  'kor2eng_analyzer':{
                                      'type':'custom',
                                      'tokenizer':'nori_tokenizer',
                                      'filter': [
                                          'trim',
                                          'lowercase',
                                          'javacafe_kor2eng'
                                      ]
                                  },
                                  'eng2kor_analyzer': {
                                      'type': 'custom',
                                      'tokenizer': 'nori_tokenizer',
                                      'filter': [
                                          'trim',
                                          'lowercase',
                                          'javacafe_eng2kor'
                                      ]
                                  },
                              },
                              'filter':{
                                  'my_shingle_f':{
                                      "type": "shingle"
                                  }
                              }
                          },
                          'similarity':{
                              'my_similarity':{
                                  'type':'BM25',
                              }
                          }
                      },
                      'mappings':{
                          'properties':{
                              'title':{
                                  'type':'keyword',
                                  'copy_to':['title_kor2eng','title_eng2kor']
                              },
                              'title_kor2eng': {
                                  'type': 'text',
                                  'analyzer':'my_analyzer',
                                  'search_analyzer': 'kor2eng_analyzer'
                              },
                              'title_eng2kor': {
                                  'type': 'text',
                                  'analyzer':'my_analyzer',
                                  'search_analyzer': 'eng2kor_analyzer'
                              },
                              'text':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity',
                              },
                              'text_origin': {
                                  'type': 'text',
                                  'analyzer': 'my_analyzer',
                                  'similarity': 'my_similarity'
                              }
                          }
                      }
                  }
                  )

In [ ]:
print(es.indices.get('document'))

In [ ]:
!wget "https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2"

In [ ]:
!python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2

In [ ]:
from glob import glob

title = []
text = []

for folder_path in glob('/home/dr_lunars/text/*'):
    for file_path in glob(folder_path+'/*'):
        f = open(file_path, 'r')
        data = f.read()
        f.close()
        for doc in data.split('</doc>'):
            doc_split = doc.split('">\n')
            try:
                if len(doc_split[1]) >= 500:
                    title.append(doc_split[0].split('" title="')[1])
                    text.append(doc_split[1])
            except:
                pass

In [ ]:
import pandas as pd

wiki_data = pd.DataFrame({'title':title,'text':text})

In [ ]:
import re

wiki_data['text_origin'] = wiki_data['text']

wiki_data['text_origin'] = wiki_data['text_origin'].apply(lambda x : ' '.join(re.sub(r'''[^ \r\nㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9ぁ-ゔァ-ヴー々〆〤一-龥~₩!@#$%^&*()“”‘’《》≪≫〈〉『』「」＜＞_+|{}:"<>?`\-=\\[\];',.\/·]''', ' ', str(x.lower().strip())).split()))

wiki_data['text'] = wiki_data['text'].apply(lambda x : x.replace('\\n\\n',' '))
wiki_data['text'] = wiki_data['text'].apply(lambda x : x.replace('\n\n',' '))
wiki_data['text'] = wiki_data['text'].apply(lambda x : x.replace('\\n',' '))
wiki_data['text'] = wiki_data['text'].apply(lambda x : x.replace('\n',' '))

wiki_data['text'] = wiki_data['text'].apply(lambda x : ' '.join(re.sub(r'''[^ \r\nㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9~₩!@#$%^&*()_+|{}:"<>?`\-=\\[\];',.\/]''', ' ', str(x.lower().strip())).split()))

In [ ]:
from tqdm import tqdm

title = []
text = []
text_origin = []

for num in tqdm(range(len(wiki_data))):
    cnt = 0
    while cnt < len(wiki_data['text'][num]):
        title.append(wiki_data['title'][num])
        text.append(wiki_data['text'][num][cnt:cnt+1000])
        text_origin.append(wiki_data['text_origin'][num])
        cnt+=1000

In [ ]:
df = pd.DataFrame({'title':title,'text':text,'text_origin':text_origin})

In [ ]:
from elasticsearch import Elasticsearch, helpers

buffer = []
rows = 0

for num in tqdm(range(len(df))):
    article = {"_id": num,
               "_index": "document", 
               "title" : df['title'][num],
               "text" : df['text'][num],
               "text_origin" : df['text_origin'][num]}

    buffer.append(article)

    rows += 1

    if rows % 3000 == 0:
        helpers.bulk(es, buffer)
        buffer = []
        print("Inserted {} articles".format(rows), end="\r")

if buffer:
    helpers.bulk(es, buffer)

print("Total articles inserted: {}".format(rows))

In [ ]:
es.indices.create(index = 'qa',
                  body = {
                      'settings':{
                          'analysis':{
                              'analyzer':{
                                  'my_analyzer':{
                                      "type": "custom",
                                      'tokenizer':'nori_tokenizer',
                                      'decompound_mode':'mixed',
                                      'stopwords':'_korean_',
                                      'synonyms':'_korean_',
                                      "filter": ["lowercase",
                                                 "my_shingle_f",
                                                 "nori_readingform",
                                                 "cjk_bigram",
                                                 "decimal_digit",
                                                 "stemmer",
                                                 "trim"]
                                  },
                                  'kor2eng_analyzer':{
                                      'type':'custom',
                                      'tokenizer':'nori_tokenizer',
                                      'filter': [
                                          'trim',
                                          'lowercase',
                                          'javacafe_kor2eng'
                                      ]
                                  },
                                  'eng2kor_analyzer': {
                                      'type': 'custom',
                                      'tokenizer': 'nori_tokenizer',
                                      'filter': [
                                          'trim',
                                          'lowercase',
                                          'javacafe_eng2kor'
                                      ]
                                  },
                              },
                              'filter':{
                                  'my_shingle_f':{
                                      "type": "shingle"
                                  }
                              }
                          },
                          'similarity':{
                              'my_similarity':{
                                  'type':'BM25',
                              }
                          }
                      },
                      'mappings':{
                          'properties':{
                              'question':{
                                  'type':'text',
                                  'copy_to':['question_kor2eng','question_eng2kor']
                              },
                              'question_kor2eng': {
                                  'type': 'text',
                                  'analyzer':'my_analyzer',
                                  'search_analyzer': 'kor2eng_analyzer'
                              },
                              'question_eng2kor': {
                                  'type': 'text',
                                  'analyzer':'my_analyzer',
                                  'search_analyzer': 'eng2kor_analyzer'
                              },
                              'answer':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity',
                              }
                          }
                      }
                  }
                  )

In [ ]:
print(es.indices.get('qa'))

In [ ]:
from datasets import load_dataset

dataset_v1 = load_dataset('squad_kor_v1')

In [ ]:
import json

with open("ko_wiki_v1_squad.json", "r") as f:
    ko_wiki_v1_squad = json.load(f)

with open("ko_nia_normal_squad_all.json", "r") as f:
    ko_nia_normal_squad_all = json.load(f)

In [ ]:
questions = dataset_v1['train']['question'] + dataset_v1['validation']['question'] + [i['paragraphs'][0]['qas'][0]['question'] for i in ko_wiki_v1_squad['data']] + [j['question'] for i in ko_nia_normal_squad_all['data'] for j in i['paragraphs'][0]['qas']]
answers = [i['text'][0] for i in dataset_v1['train']['answers']] + [i['text'][0] for i in dataset_v1['validation']['answers']] + [i['paragraphs'][0]['qas'][0]['answers'][0]['text'] for i in ko_wiki_v1_squad['data']] + [j['answers'][0]['text'] for i in ko_nia_normal_squad_all['data'] for j in i['paragraphs'][0]['qas']]

In [ ]:
import pandas as pd

df = pd.DataFrame({'question':questions,'answer':answers})

In [ ]:
from tqdm import tqdm
from elasticsearch import Elasticsearch, helpers

buffer = []
rows = 0

for num in tqdm(range(len(df))):
    article = {"_id": num,
               "_index": "qa", 
               "question" : df['question'][num],
               "answer" : df['answer'][num]}

    buffer.append(article)

    rows += 1

    if rows % 3000 == 0:
        helpers.bulk(es, buffer)
        buffer = []
        print("Inserted {} articles".format(rows), end="\r")

if buffer:
    helpers.bulk(es, buffer)

print("Total articles inserted: {}".format(rows))